In [3]:
# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import logging
import subprocess
from multiprocessing.pool import ThreadPool
import joblib

# SCikitlearn modules :
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report , roc_auc_score

# Scipy modules : 
from scipy.stats import fisher_exact

***

In [2]:
# Make the blastp DB of all the dpo sequences :

path_seqbased = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model"

fasta_file = f"{path_seqbased}/cdhit_clusters_2912/0.75.out"

blast_command = f"makeblastdb -in {fasta_file} -dbtype prot -out {path_seqbased}/TropiSeq/TropiSeq_0.75.db"
make_blast_process = subprocess.Popen(blast_command, shell =True, stdout = subprocess.PIPE, stderr=subprocess.STDOUT)
mkblast_out, mkblast_err = make_blast_process.communicate()
print(mkblast_out , mkblast_err)

b'\n\nBuilding a new DB, current time: 01/01/2024 18:47:52\nNew DB name:   /media/concha-eloko/Linux/PPT_clean/Seqbased_model/TropiSeq/TropiSeq_0.75.db\nNew DB title:  /media/concha-eloko/Linux/PPT_clean/Seqbased_model/cdhit_clusters_2912/0.75.out\nSequence type: Protein\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 883 sequences in 0.0848188 seconds.\n' None


***
# Make function that :
### A : blastp from a Dpo seq
### B : read the results and spot the hits
### C : Build a vector from the presence abscence
### D : Make prediction
***

> 77 phages

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model/dico_cluster.cdhit__0.8.json \
/media/concha-eloko/Linux/PPT_clean/Seqbased_model

In [1]:
import json

path_seqbased = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model"
path_db = f"{path_seqbased}/TropiSeq/TropiSeq_0.75.db"

dico_cluster = json.load(open(f"{path_seqbased}/dico_cluster.cdhit__0.75.json"))
dico_cluster_r = {ref_dpo : key_dpo for key_dpo,list_dpo in dico_cluster.items() for ref_dpo in list_dpo}


In [4]:
path_seq = "/media/concha-eloko/Linux/77_strains_phage_project/rbp_work"
path_tmp =  "/media/concha-eloko/Linux/PPT_clean/Seqbased_model/tmp"
labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_seq}/77_phages_Dpo_domains.2512.multi.fasta", "fasta") if len(record.seq) >0}
set_records = [record for record in SeqIO.parse(f"{path_seq}/77_phages_Dpo_domains.2512.multi.fasta", "fasta") if len(record.seq) > 0]


def tmp_fasta_file(record , path_tmp) :
    name_file = "_".join(record.description.split(" "))
    path_fasta = f"{path_tmp}/{name_file}.fasta"
    length_seq = len(record.seq)
    with open(path_fasta, "w") as outfile :
        outfile.write(f">{record.description}\n{str(record.seq)}")
    return path_fasta , length_seq

def blast_seq(path_fasta, path_DB, path_tmp) :
    file_name = path_fasta.split("/")[-1]
    command = f"blastp -query {path_fasta} -db {path_DB} -out {path_tmp}/{file_name}.blast_out -outfmt 6 -evalue 1e-10"
    blastp_sub = subprocess.Popen(command ,shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
    out , err = blastp_sub.communicate()
    return f"{path_tmp}/{file_name}.blast_out"

def get_best_candidate(path_blast_out, length_seq, bitscore = 75) : 
    winner = 0
    labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
    blast_df = pd.read_csv(path_blast_out, sep = "\t", names = labels_blast)
    if len(blast_df) > 0 :
        row = blast_df.iloc[0] 
        if row["bitscore"] > bitscore and length_seq/int(row["length"])> 0.8:
            winner = dico_cluster_r[row["sseqid"]]
        else :
            winner = "No hits"
    else :
        winner = "No hits"
    return winner

def get_winner(record , path_tmp) :
    path_func , len_func = tmp_fasta_file(record, path_tmp)
    path_blast_out_func = blast_seq(path_func , path_db, path_tmp)
    winner = get_best_candidate(path_blast_out_func, len_func)
    return winner

> Ferriol et al 

In [5]:
ferriol_winners = []
for record in tqdm(set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        vector = [0]*len(dico_cluster)
        vector[hit] = 1
        vector = np.array(vector)
        a = (record.description , winner, vector)
    else :
        vector = "Null"
    a = (record.description , winner, vector)
    ferriol_winners.append(a)

100%|█████████████████████████████████████████| 126/126 [00:06<00:00, 18.49it/s]


> Bea et al

In [6]:
bea_winners = []

path_bea = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
path_domains_bea = f"{path_bea}/DepoScope_predictions.bea.domains.0709.fasta"

bea_dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_domains_bea}", "fasta") if len(record.seq) >0}
bea_set_records = [record for record in SeqIO.parse(f"{path_domains_bea}", "fasta") if len(record.seq) > 0]

for record in tqdm(bea_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        vector = [0]*len(dico_cluster)
        vector[hit] = 1
        vector = np.array(vector)
        a = (record.description , winner, vector)
    else :
        vector = "Null"
    a = (record.description , winner, vector)
    bea_winners.append(a)
    

100%|███████████████████████████████████████████| 71/71 [00:03<00:00, 21.58it/s]


> Towndsend et al 

In [7]:
towndsend_winners = []

path_towndsend = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Townsed"
path_domains_towndsend = f"{path_towndsend}/DepoScope_predictions.Townsed.domains.0909.fasta"

towndsend_dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_domains_towndsend}", "fasta") if len(record.seq) >0}
towndsend_set_records = [record for record in SeqIO.parse(f"{path_domains_towndsend}", "fasta") if len(record.seq) > 0]

for record in tqdm(towndsend_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        vector = [0]*len(dico_cluster)
        vector[hit] = 1
        vector = np.array(vector)
        a = (record.description , winner, vector)
    else :
        vector = "Null"
    a = (record.description , winner, vector)
    towndsend_winners.append(a)

100%|███████████████████████████████████████████| 44/44 [00:01<00:00, 24.22it/s]


> Others

In [8]:
other_winners = []

path_other = "/media/concha-eloko/Linux/PPT_clean/in_vitro"
path_domains_others = f"{path_other}/Others_all.dpos_domains.multi.fasta"

other_dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_domains_others}", "fasta") if len(record.seq) >0}
other_set_records = [record for record in SeqIO.parse(f"{path_domains_others}", "fasta") if len(record.seq) > 0]

for record in tqdm(other_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        vector = [0]*len(dico_cluster)
        vector[hit] = 1
        vector = np.array(vector)
        a = (record.description , winner, vector)
    else :
        vector = "Null"
    a = (record.description , winner, vector)
    other_winners.append(a)



100%|███████████████████████████████████████████| 56/56 [00:02<00:00, 20.32it/s]


In [10]:
other_winners

[('MZ322895.1_prot_QWY13631.1_35',
  'Dpo_cdhit_231',
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

***
# Make predictions

In [9]:
import pickle
import os
from joblib import load

path_seqbased = "/media/concha-eloko/Linux/PPT_clean"

models_TropiSeq = {}

for rf_model in os.listdir(f"{path_seqbased}/selected_RF_3112") :
        kltype = rf_model.split("_RF_")[1].split(".")[0]
        with open(f"{path_seqbased}/selected_RF_3112/{rf_model}", 'rb') as file:
            models_TropiSeq[kltype] = load(file)

TropiSeq_results = {}


> Get other predictions : 

In [10]:
# Others part : 
for _,winner in tqdm(enumerate(other_winners)) :
    if isinstance(winner[2], np.ndarray):
        tmp_positif = {}
        for kltype in models_TropiSeq :
            pred = models_TropiSeq[kltype].predict_proba(np.array(winner[2]).reshape(1, -1))
            if pred[0][1] >= 0.5 :
                tmp_positif[kltype] = pred[0][1]
        TropiSeq_results[winner[0]] = tmp_positif
    else : 
        TropiSeq_results[winner[0]] = "No hits"

56it [01:03,  1.13s/it]


In [9]:
TropiSeq_results

{'MZ322895.1_prot_QWY13631.1_35': {},
 'ON146449.1_prot_UPW35138.1_1': {'KL13': 0.6586180102243417,
  'KL2': 0.7822096035934847},
 'ON146449.1_prot_UPW35150.1_13': {'KL102': 0.6905486808781712},
 'MZ571831.1_prot_UEW68236.1_80': 'No hits',
 'MZ612130.1_prot_QYC51043.1_10': {},
 'MT197175.1_prot_QIW86415.1_45': {'KL102': 0.6905486808781712},
 'MT197175.1_prot_QIW86419.1_49': {'KL110': 0.5504562299259087,
  'KL9': 0.6399021972544233,
  'KL106': 0.6856481893629041},
 'MT197175.1_prot_QIW86428.1_58': {},
 'MZ571832.1_prot_UEP19662.1_17': 'No hits',
 'MZ571832.1_prot_UEP19667.1_22': {'KL102': 0.6905486808781712},
 'MZ571834.1_prot_UEP19705.1_4': {'KL21': 0.8550435757065198,
  'KL39': 0.681684737764896,
  'KL166': 0.5277423924804487,
  'KL24': 0.8107096548933331,
  'KL28': 0.6058884872943476,
  'KL48': 0.8725,
  'KL112': 0.92117428052397},
 'ON881905.1_prot_UTN90143.1_15': 'No hits',
 'MT197176.1_prot_QJI52618.1_44': 'No hits',
 'MT197176.1_prot_QJI52623.1_49': {'KL110': 0.5504562299259087,


In [10]:
import pandas as pd
path_other = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Others"

other_info = pd.read_csv(f"{path_other}/index_others.tsv", sep = "\t", names = ["index_A", "index_B", "prot_names"])

other_TropiSeq_results = {}
for prot_id, results in TropiSeq_results.items() : 
    index_a, index_b = int(prot_id.split("__")[0]) , int(prot_id.split("__")[1])
    real_name = other_info[(other_info["index_A"] == index_a) & (other_info["index_B"] == index_b)]["prot_names"].values[0]
    other_TropiSeq_results[real_name] = results

    
    

ValueError: invalid literal for int() with base 10: 'MZ322895.1_prot_QWY13631.1_35'

In [15]:
other_info

,index_A,index_B,prot_names
0,1,49,NC_029099.1_prot_YP_009226011.1_50
1,3,1,MW655991.1_prot_QUU29414.1_2
2,5,54,AB897757.1_prot_BAQ02835.1_55
3,5,55,AB897757.1_prot_BAQ02836.1_56
4,5,56,AB897757.1_prot_BAQ02837.1_57
5,5,57,AB897757.1_prot_BAQ02838.1_58
6,5,58,AB897757.1_prot_BAQ02839.1_59
7,5,59,AB897757.1_prot_BAQ02840.1_60
8,5,60,AB897757.1_prot_BAQ02841.1_61
9,5,61,AB897757.1_prot_BAQ02842.1_62


In [11]:
other_TropiSeq_results = TropiSeq_results

In [18]:
sorted_dict = {k: other_TropiSeq_results[k] for k in sorted(other_TropiSeq_results)}
sorted_dict
for prot in sorted_dict :
    if isinstance(sorted_dict[prot], dict) :
        print(prot , "; ".join([str(kl) +":"+ str(round(pred,5)) for kl,pred in sorted_dict[prot].items()]) , sep = "\t" )
    elif isinstance(sorted_dict[prot], str) :
        print(prot , "No hits", sep = "\t")
    else :
        pass
        #print(prot, "", sep = "\t")




AB716666.1_prot_BAP15736.1_24	KL102:0.69055
AB716666.1_prot_BAP15746.1_34	No hits
AB897757.1_prot_BAQ02835.1_55	
AB897757.1_prot_BAQ02836.1_56	KL70:1.0; KL21:0.94915
AB897757.1_prot_BAQ02837.1_57	KL123:0.7869
AB897757.1_prot_BAQ02838.1_58	No hits
AB897757.1_prot_BAQ02839.1_59	KL25:0.5936
AB897757.1_prot_BAQ02840.1_60	KL35:0.51355
AB897757.1_prot_BAQ02841.1_61	No hits
AB897757.1_prot_BAQ02842.1_62	KL64:0.93193
AB897757.1_prot_BAQ02843.1_63	KL30:0.89789
AB897757.1_prot_BAQ02844.1_64	No hits
MH844531.1_prot_AYP28213.1_32	No hits
MK903728.1_prot_QDF14639.1_37	KL102:0.69055
MK903728.1_prot_QDF14644.1_42	KL47:0.95161
MK903728.1_prot_QDF14645.1_43	KL47:0.91516
MN781108.1_prot_QGZ15323.1_262	
MT197175.1_prot_QIW86415.1_45	KL102:0.69055
MT197175.1_prot_QIW86419.1_49	KL110:0.55046; KL9:0.6399; KL106:0.68565
MT197175.1_prot_QIW86428.1_58	
MT197176.1_prot_QJI52618.1_44	No hits
MT197176.1_prot_QJI52623.1_49	KL110:0.55046; KL9:0.6399; KL106:0.68565
MT197176.1_prot_QJI52632.1_58	KL13:0.57088; KL2:0.8

In [21]:
sorted_dict

{'AB716666.1_prot_BAP15736.1_24': {'KL102': 0.6905486808781712},
 'AB716666.1_prot_BAP15746.1_34': 'No hits',
 'AB897757.1_prot_BAQ02835.1_55': {},
 'AB897757.1_prot_BAQ02836.1_56': {'KL70': 1.0, 'KL21': 0.9491472402816709},
 'AB897757.1_prot_BAQ02837.1_57': {'KL123': 0.7868957892450444},
 'AB897757.1_prot_BAQ02838.1_58': 'No hits',
 'AB897757.1_prot_BAQ02839.1_59': {'KL25': 0.593604028037223},
 'AB897757.1_prot_BAQ02840.1_60': {'KL35': 0.5135538858293371},
 'AB897757.1_prot_BAQ02841.1_61': 'No hits',
 'AB897757.1_prot_BAQ02842.1_62': {'KL64': 0.9319302667291399},
 'AB897757.1_prot_BAQ02843.1_63': {'KL30': 0.8978944276582059},
 'AB897757.1_prot_BAQ02844.1_64': {},
 'MH844531.1_prot_AYP28213.1_32': 'No hits',
 'MK903728.1_prot_QDF14639.1_37': {'KL102': 0.6905486808781712},
 'MK903728.1_prot_QDF14644.1_42': {'KL47': 0.9516084799575152},
 'MK903728.1_prot_QDF14645.1_43': {'KL47': 0.9151637668488338},
 'MN781108.1_prot_QGZ15323.1_262': {},
 'MT197175.1_prot_QIW86415.1_45': {'KL102': 0.6905

> Get predictions :

In [22]:
# Ferriol part : 
for _,winner in tqdm(enumerate(ferriol_winners)) :
    if isinstance(winner[2], np.ndarray):
        tmp_positif = {}
        for kltype in models_TropiSeq :
            pred = models_TropiSeq[kltype].predict_proba(np.array(winner[2]).reshape(1, -1))
            if pred[0][1] >= 0.5 :
                tmp_positif[kltype] = pred[0][1]
        TropiSeq_results[winner[0]] = tmp_positif
    else : 
        TropiSeq_results[winner[0]] = "No hits"

# Bea part : 
for _,winner in tqdm(enumerate(bea_winners)) :
    if isinstance(winner[2], np.ndarray):
        tmp_positif = {}
        for kltype in models_TropiSeq :
            pred = models_TropiSeq[kltype].predict_proba(np.array(winner[2]).reshape(1, -1))
            if pred[0][1] >= 0.5 :
                tmp_positif[kltype] = pred[0][1]
        TropiSeq_results[winner[0]] = tmp_positif
    else : 
        TropiSeq_results[winner[0]] = "No hits"

# Towndsend part :
for _,winner in tqdm(enumerate(towndsend_winners)) :
    if isinstance(winner[2], np.ndarray):
        tmp_positif = {}
        for kltype in models_TropiSeq :
            pred = models_TropiSeq[kltype].predict_proba(np.array(winner[2]).reshape(1, -1))
            if pred[0][1] >= 0.5 :
                tmp_positif[kltype] = pred[0][1]
        TropiSeq_results[winner[0]] = tmp_positif
    else : 
        TropiSeq_results[winner[0]] = "No hits"


with open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model.results.bit75.1001.tsv" , "w") as outfile :
    for prot in TropiSeq_results :
        if TropiSeq_results[prot] == "No hits" :
            outfile.write(f"{prot}\tNo hits\n")
        elif len(TropiSeq_results[prot]) == 0 :
            outfile.write(f"{prot}\tNo hits\n")
        else :
            outfile.write(f"{prot}\t")
            hits = [f"{kltype}:{round(score,3)}" for kltype, score in TropiSeq_results[prot].items()]
            outfile.write(";".join(hits))
            outfile.write("\n")

126it [03:36,  1.72s/it]
71it [02:49,  2.39s/it]
44it [01:18,  1.79s/it]


In [25]:
with open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model.results.bit75.tsv" , "w") as outfile :
    for prot in TropiSeq_results :
        if TropiSeq_results[prot] == "No hits" :
            outfile.write(f"{prot}\tNo hits\n")
        elif len(TropiSeq_results[prot]) == 0 :
            outfile.write(f"{prot}\tNo hits\n")
        else :
            outfile.write(f"{prot}\t")
            hits = [f"{kltype}:{round(score,3)}" for kltype, score in TropiSeq_results[prot].items()]
            outfile.write(" ; ".join(hits))
            outfile.write("\n")

In [23]:
import pprint
pp = pprint.PrettyPrinter(width = 150, sort_dicts = True, compact = True)
out = pp.pprint(TropiSeq_results)


{'A1a_00002': {'KL102': 0.6905486808781712},
 'A1a_00014': {'KL151': 0.6979182642123104},
 'A1b_00036': {'KL102': 0.6905486808781712},
 'A1b_00048': {'KL157': 0.7285848965848967},
 'A1c_00034': 'No hits',
 'A1c_00046': {'KL102': 0.6905486808781712},
 'A1d_00009': {'KL112': 0.9662806497302906},
 'A1d_00013': {'KL102': 0.6905486808781712},
 'A1e_00012': {'KL112': 0.92117428052397,
               'KL166': 0.5277423924804487,
               'KL21': 0.8550435757065198,
               'KL24': 0.8107096548933331,
               'KL28': 0.6058884872943476,
               'KL39': 0.681684737764896,
               'KL48': 0.8725},
 'A1e_00024': {'KL102': 0.6905486808781712},
 'A1f_00012': {'KL112': 0.92117428052397,
               'KL166': 0.5277423924804487,
               'KL21': 0.8550435757065198,
               'KL24': 0.8107096548933331,
               'KL28': 0.6058884872943476,
               'KL39': 0.681684737764896,
               'KL48': 0.8725},
 'A1f_00024': {'KL102': 0.69054868087

In [80]:
TropiSeq_results = {'K10PH82C1__cds_50_A_5_301_819.pdb': 'No hits',
 'K10PH82C1__cds_51_A_2_38_368.pdb': {},
 'K11PH164C1__cds_45_A_5_356_700.pdb': 'No hits',
 'K11PH164C1__cds_46_A_1_1_416.pdb': {},
 'K13PH07C1S__cds_10_A_7_32_375.pdb': 'No hits',
 'K13PH07C1S__cds_11_A_2_93_430.pdb': 'No hits',
 'K14PH164C1__cds_24_A_4_221_871.pdb': {},
 'K15PH90__cds_55_A.pdb': {'KL136': 1, 'KL15': 1},
 'K16PH164C3__cds_48_A_3_292_776.pdb': {'KL16': 1},
 'K17alfa61__cds_23_A_4_179_630.pdb': 'No hits',
 'K17alfa62__cds_64_A_3_129_548.pdb': {'KL17': 1},
 'K1PH164C1__cds_8_A_2_69_559.pdb': 'No hits',
 'K21lambda1__cds_28_A.pdb': {'KL124': 1, 'KL125': 1, 'KL21': 1, 'KL30': 1, 'KL31': 1, 'KL39': 1},
 'K22PH164C1__cds_10_A_1_1_368.pdb': {'KL111': 1},
 'K23PH08C2__cds_233_A_2_76_514.pdb': {'KL23': 1},
 'K24PH164C1__cds_8_A_2_85_402.pdb': {'KL112': 1, 'KL19': 1, 'KL21': 1, 'KL24': 1, 'KL28': 1, 'KL39': 1},
 'K25PH129C1__cds_60_A_5_311_671.pdb': {'KL25': 1},
 'K26PH128C1__cds_49_A_3_291_808.pdb': {'KL74': 1},
 'K26PH128C1__cds_50_A_1_97_595.pdb': {'KL24': 1},
 'K27PH129C1__cds_48_A_7_200_648.pdb': {'KL20': 1, 'KL27': 1},
 'K2PH164C1__cds_23_A_6_269_664.pdb': 'No hits',
 'K2PH164C2__cds_24_A_6_274_608.pdb': {'KL2': 1},
 'K2alfa62__cds_23_A_6_269_671.pdb': 'No hits',
 'K35PH164C3__cds_48_A_4_282_728.pdb': {},
 'K37PH164C1__cds_47_A_1_1_307.pdb': {'KL111': 1},
 'K37PH164C1__cds_48_A_2_37_367.pdb': {'KL2': 1},
 'K38PH09C2__cds_24_A_4_178_672.pdb': {'KL38': 1},
 'K39PH122C2__cds_55_A_5_286_658.pdb': {'KL106': 1, 'KL110': 1},
 'K39PH122C2__cds_8_A_4_33_387.pdb': 'No hits',
 'K40PH129C1__cds_56_A_4_239_860.pdb': 'No hits',
 'K41P2__cds_11_A_7_188_506.pdb': {'KL60': 1},
 'K43PH164C1__cds_40_A_3_296_718.pdb': {'KL102': 1},
 'K43PH164C1__cds_41_A_3_32_385.pdb': {'KL145': 1, 'KL18': 1},
 'K44PH129C1__cds_10_A_3_35_538.pdb': {'KL142': 1},
 'K44PH129C1__cds_9_A_2_39_391.pdb': {'KL111': 1},
 'K45PH128C2__cds_237_A_3_76_463.pdb': {'KL45': 1},
 'K45PH128C2__cds_239_A_1_186_411.pdb': {'KL63': 1},
 'K46PH129__cds_24_A_8_222_596.pdb': {'KL122': 1, 'KL136': 1},
 'K48PH164C1__cds_49_A_3_306_628.pdb': {'KL128': 1, 'KL22': 1},
 'K4PH164__cds_22_A_1_53_402.pdb': {},
 'K51PH129C1__cds_9_A_1_92_787.pdb': {'KL51': 1},
 'K53PH164C2__cds_24_A_4_200_777.pdb': {},
 'K54lambda1_1_1__cds_238_A_1_78_646.pdb': {'KL24': 1},
 'K54lambda2__cds_23_A_7_214_582.pdb': 'No hits',
 'K56PH164C1__cds_48_A_5_293_702.pdb': 'No hits',
 'K56PH164C1__cds_49_A_3_110_475.pdb': {'KL56': 1},
 'K57lambda1_2__cds_92_A_3_120_506.pdb': {},
 'K57lambda1_2__cds_93_A_4_251_900.pdb': {},
 'K58PH129C2__cds_47_A_2_38_516.pdb': {},
 'K5lambda5__cds_196_A_4_457_808.pdb': {'KL145': 1, 'KL18': 1},
 'K5lambda5__cds_198_A_3_173_674.pdb': {'KL27': 1},
 'K5lambda5__cds_199_A_2_109_658.pdb': 'No hits',
 'K5lambda5__cds_200_A_2_111_443.pdb': {'KL116': 1},
 'K60PH164C1__cds_94_A_3_119_467.pdb': {'KL145': 1, 'KL18': 1},
 'K60PH164C1__cds_96_A_6_321_703.pdb': {'KL71': 1},
 'K61PH164C1__cds_10_A_3_80_443.pdb': {'KL151': 1},
 'K61PH164C1__cds_9_A_1_1_387.pdb': {},
 'K63PH128__cds_22_A_5_260_714.pdb': {'KL63': 1},
 'K64PH164C4__cds_24_A_4_178_852.pdb': {'KL64': 1},
 'K65PH164__cds_12_A_5_181_627.pdb': {'KL60': 1},
 'K66PH128C1__cds_59_A_4_252_713.pdb': {'KL66': 1},
 'K6PH25C3__cds_23_A_3_206_691.pdb': 'No hits',
 'K71PH129C1__cds_55_A_5_295_682.pdb': {'KL45': 1},
 'K74PH129C2__cds_51_A_3_291_808.pdb': {'KL74': 1},
 'K74PH129C2__cds_52_A_2_34_431.pdb': {'KL43': 1},
 'K80PH1317a__cds_52_A_5_302_576.pdb': {'KL74': 1},
 'K80PH1317a__cds_53_A.pdb': {'KL74': 1},
 'K80PH1317a__cds_54_A_2_36_354.pdb': {'KL15': 1},
 'K80PH1317b__cds_52_A_5_302_606.pdb': {'KL74': 1},
 'K80PH1317b__cds_53_A.pdb': 'No hits',
 'K80PH1317b__cds_54_A_2_36_354.pdb': {'KL15': 1},
 'K82P1__cds_45_A_5_292_704.pdb': 'No hits',
 'K82P1__cds_46_A_2_94_449.pdb': 'No hits',
 'K8PH128__cds_46_A_5_294_719.pdb': 'No hits'}

In [24]:

with open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model.1001.results.tsv" , "w") as outfile :
    for prot in TropiSeq_results :
        if TropiSeq_results[prot] == "No hits" :
            outfile.write(f"{prot}\tNo hits\n")
        elif len(TropiSeq_results[prot]) == 0 :
            outfile.write(f"{prot}\tNo hits\n")
        else :
            outfile.write(f"{prot}\t")
            hits = [f"{kltype}: {round(score, 3)}" for kltype, score in TropiSeq_results[prot].items()]
            outfile.write(";".join(hits))
            outfile.write("\n")
                

# 